# L0 data clean

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import imp
import datetime
import pytz
%matplotlib notebook

In [3]:
#input path of new file to clean
#time period should overlap with that of previous file; no gaps.
Glacier= 'Wolverine'
Station= '990'
#file= "wolverine990_2017_0911.txt"
file='gulkana_1480_allrawdata.csv'
folder=r"Q:\Project Data\GlacierData\Benchmark_Program\Data\Gulkana\AllYears\Wx\Raw"
pth=os.path.join(folder, file)
out_date_format='%Y/%m/%d %H:%M'

In [4]:
# if Glacier + Station== r'Wolverine990':
#     header_rws=2 #number of header rows to not read in
#     col_nms=["Date", "Time", "Instrument", "Value", "Unit", "Flag"]
    
# if Glacier + Station==r'wolverine1420':
#     header_rws=4 #number of header rows to not read in
#     #Grab correct column names (a few rows up in Campbell Logger output)
#     coltable=pd.read_csv(pth, header=1)
#     col_nms=coltable.columns

In [5]:
# #Read in data
# dat=pd.read_csv(pth, header=header_rws)
# dat.columns= col_nms #Name columns

In [6]:
#Set constants
date_format='%m/%d/%Y %H:%M:%S'
timezone='America/Anchorage' #choose from pytz.all_timezones

In [7]:
# for col in list(dat):
#     print ("stripping whitespace from " + col) #print column name    
#     if type(dat[col].values[0]).__name__ =='str': #if column is a string, strip white space
#         dat[col]=dat[col].str.strip()

In [8]:
dat=pd.read_csv("Q:\Project Data\GlacierData\Benchmark_Program\Data\Wolverine\AllYears\Wx\Raw\wolverine990_allrawdata.csv")

C:\Users\ehbaker\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
dat.tail()

,Date,Time,Instrument,Value,Unit,Flag,DateTime
2952296,09/10/2017,17:30:00,PC,0.566,m,G,09/10/2017 17:30:00
2952297,09/10/2017,17:30:00,RH,93.000,%,G,09/10/2017 17:30:00
2952298,09/10/2017,17:30:00,AT,5.200,C,G,09/10/2017 17:30:00
2952299,09/10/2017,17:30:00,AT2,5.100,C,G,09/10/2017 17:30:00
2952300,09/10/2017,17:30:00,B,-0.582,kPa,G,09/10/2017 17:30:00


In [9]:
#START HERE IF YOU HAVE external dataframe form large concatenated version
#dat=pd.read_csv(r'Q:/Project Data/GlacierData/Benchmark_Program/Data/Gulkana/AllYears/Wx/Raw/wolverine_990_allrawdata.csv')

In [10]:
dat.head()

,Date,Time,Instrument,Value,Unit,Flag,DateTime
0,03/01/2010,23:15:00,WindDir,900.0,Deg,G,03/01/2010 23:15:00
1,03/01/2010,23:15:00,PrecipStage,0.0,M,B,03/01/2010 23:15:00
2,03/01/2010,23:15:00,WindSpeed,0.9,m/s,G,03/01/2010 23:15:00
3,03/01/2010,23:28:00,WindSpeed,0.4,m/s,G,03/01/2010 23:28:00
4,03/01/2010,23:28:00,WindDir,914.0,Deg,G,03/01/2010 23:28:00


In [11]:
#Fix time
#Create date-time column
#dat['DateTime']=dat.Date+ " " + dat.Time #this is needed for non-cumulative processing
dat.loc[:,'DateTime']=pd.to_datetime(dat['DateTime'], format=date_format) #set to date-time from string
#Correct for time slips
dat['DateTime']=dat['DateTime'].dt.round('15min') #round time to the nearest 15 minute value
#Convert to UTC time
dat['UTC_col']=dat.DateTime + datetime.timedelta(hours=8)
dat['UTC_col'].timezone='UTC'
#Create column for local time
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)

In [12]:
dat=dat.set_index('UTC_col')
#Create column for local time
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)

In [13]:
#Create local time
dat['local_time'] = dat.index.tz_localize('UTC').tz_convert(local_timezone)

In [14]:
dat['index_local_time']=dat.local_time.copy()

In [15]:
#Set index to local time
dat=dat.set_index('index_local_time')

In [16]:
dat=dat.drop_duplicates(subset=['DateTime', 'Instrument'], keep='first') #gets rid of overlaps (i.e. multiple downloads of same bit of data)

In [17]:
#Wrote into a one-liner with "drop_duplicates" above; keep this in case any issues arise in trouble-shooting
# #In the uncommon instance that a sensor has duplicate values at a single timestep, set sensor name and value to NAN
# # - this occurs occasionally during maintenance, as sensors are being renamed and added
# for ii in dat.local_time.unique():
#     testdat=dat[ii:ii]
#     if len(testdat[testdat.Instrument.duplicated()])>0:
#         duplicate_sensor_name=testdat.loc[testdat.Instrument.duplicated(), 'Instrument'][0]
#         print(duplicate_sensor_name + " is being set to NAN at " + str(ii))
#         dat.loc[(dat.Instrument==duplicate_sensor_name) & (dat.index==ii), 'Instrument']=np.nan
#         dat.loc[(dat.Instrument==duplicate_sensor_name) & (dat.index==ii), 'Value']=np.nan

In [18]:
#Set time subset you are wanting here
#dat=dat['2015-09-20':]

In [19]:
dat.tail(2)

,Date,Time,Instrument,Value,Unit,Flag,DateTime,local_time
index_local_time,,,,,,,,
2017-09-10 17:30:00-08:00,09/10/2017,17:30:00,AT2,5.100,C,G,2017-09-10 17:30:00,2017-09-10 17:30:00-08:00
2017-09-10 17:30:00-08:00,09/10/2017,17:30:00,B,-0.582,kPa,G,2017-09-10 17:30:00,2017-09-10 17:30:00-08:00


In [20]:
if Glacier + Station == "Gulkana1480":
    dat.loc[dat.Instrument=='PC', 'Instrument']="TPG" #TPG gage labeled as both PC and TPG; combine
    dat.loc[(dat.Instrument=='WD') & (dat.index>'2015-09-25 12'), 'Instrument']='WDSA' #before this date, "WD" was simply wind direction; avfter, is vector-averaged
if Glacier + Station=="Wolverine990":
    dat.loc[(dat.Instrument=='WD') & (dat.index>'2015-09-06 17:15'), 'Instrument']='WDSA' #before this date, "WD" was simply wind direction; avfter, is vector-averaged

In [21]:
if Glacier + Station =='Wolverine990':  
    #Name recorded on the logger changed thru time; indicate which are grouped (i.e. logging same variable) here
    wind_gust_labels=['WindGusts', 'WG', 'WSG']
    wind_speed_labels=['AvgWind', 'WS']
    wind_dir_labels=['WindDir', 'WD']
    temp_unasp1_labels=['AirTemp1', 'T1', 'T']
    temp_unasp2_labels=['AirTemp2', 'T2']
    temp_asp1_labels=['AirTempAsp', 'AspT', 'AT']
    temp_asp2_labels=['ASPTRH', 'AT2']
    precip_stage_labels=['PR']
    precip_weighing_labels=['TPG', 'PC']
    rel_hum_labels=['RH']
    barometer_labels=['B']
    logger_bat_label=['BV']
    logger_temp_label=['Tinternal']
    bad_labels=['PrecipStage', 'WindSpeed', 'AirTemp3', 'Air Temp w/FanN', 'AirTemp w/Fan', 'VectorAvgWindDir', 'PS', 'MWD', 'WDA'] #labels containing only bad data; introduced during maintenence etc.
    
    #Rename instruments
    dat.loc[dat.Instrument.isin(wind_gust_labels), "Instrument"]='WindGustSpeed'
    dat.loc[dat.Instrument.isin(wind_speed_labels), "Instrument"]='WindSpeed'
    dat.loc[dat.Instrument.isin(wind_dir_labels), "Instrument"]='WindDir'
    dat.loc[dat.Instrument.isin(temp_unasp1_labels), "Instrument"]='Tpassive1'
    dat.loc[dat.Instrument.isin(temp_unasp2_labels), "Instrument"]='Tpassive2'
    dat.loc[dat.Instrument.isin(temp_asp1_labels), "Instrument"]='TAspirated1'
    dat.loc[dat.Instrument.isin(temp_asp2_labels), "Instrument"]='TAspirated2'
    dat.loc[dat.Instrument.isin(precip_stage_labels), "Instrument"]='StageCumulative'
    dat.loc[dat.Instrument.isin(precip_weighing_labels), "Instrument"]='TPGCumulative'
    dat.loc[dat.Instrument.isin(rel_hum_labels), "Instrument"]='RelHum'
    dat.loc[dat.Instrument.isin(barometer_labels), "Instrument"]='Barom'
    dat.loc[dat.Instrument.isin(logger_bat_label), "Instrument"]='LoggerBattery'
    dat.loc[dat.Instrument.isin(logger_temp_label), "Instrument"]='LoggerTemp'
    
    #Rename logger variables
    dat.loc[dat.Instrument.isin(rel_hum_labels), "Instrument"]='RelHum'

    #Remove rows with bad data
    dat=dat[~dat.Instrument.isin(bad_labels)] #Remove where labels are bad; introduced during maintenance

In [22]:
#Cast the data from long -> wide format
dat=dat.reset_index() #reset index so all unique
wide_dat=dat.pivot_table(columns='Instrument', values='Value', index='local_time')

In [23]:
wide_dat.tail(1)

Instrument,Barom,LoggerBattery,LoggerTemp,RelHum,Stage,StageCumulative,TAspirated1,TAspirated2,TPGCumulative,Tpassive1,Tpassive2,WDSA,WindDir,WindGustSpeed
local_time,,,,,,,,,,,,,,
2017-09-10 17:30:00-08:00,-0.582,13.8,10.2,93.0,NaN,NaN,5.2,5.1,0.566,5.0,NaN,90.0,NaN,3.0


In [24]:
# #Rename columns to match Shad's processing
# if Glacier + Station =="Wolverine990":
#     new_cols=['TAspirated1','TAspirated2','AtmPressure',
#                'LoggerBattery', 'TPGCumulative', 'RelHum',
#               'Tpassive1', 'LoggerTemp',
#               'WindDir', 'WindSpeed',  'WindGustSpeed']
# if Glacier + Station== "Gulkana1480":
#     new_cols=['TAspirated1','TAspirated2','LoggerBattery',
#                'StageCumulative','RelHum',
#               'Tpassive1', 'TPGCumulative', 'LoggerTemp',
#               'WindDir', 'VecAvgWindSpeed', 'WindSpeed',  'WindGustSpeed']

In [25]:
#wide_dat.columns=new_cols

#SOON - calculate the vector average wind speed HERE (should be done on logger; thus, appropriate L0 edit)

In [26]:
out_columns=['UTC_time', 'AK_time', 'Tpassive1', 'Tpassive2',
       'TAspirated1', 'TAspirated2', 'RelHum', 'StageCumulative',
       'TPGCumulative', 'WindSpeed', 'WindGustSpeed', 'WindDir', 'LoggerTemp',
       'LoggerBattery']

In [27]:
for col in out_columns:
    if col not in wide_dat.columns:
        wide_dat[col]=np.nan #create the column; fill with NANs

In [28]:
wide_dat['UTC_time']=wide_dat.index.tz_convert('UTC').strftime(out_date_format)#Create column for true local time (as string, not UTC - X hrs)

In [29]:
wide_dat['AK_time']=wide_dat.index.tz_convert('America/Anchorage').strftime(out_date_format)#Create column for true local time (as string, not UTC - X hrs)

In [30]:
wide_dat.tail()

Instrument,Barom,LoggerBattery,LoggerTemp,RelHum,Stage,StageCumulative,TAspirated1,TAspirated2,TPGCumulative,Tpassive1,Tpassive2,WDSA,WindDir,WindGustSpeed,UTC_time,AK_time,WindSpeed
local_time,,,,,,,,,,,,,,,,,
2017-09-10 16:30:00-08:00,-0.606,13.9,8.8,94.0,NaN,NaN,5.0,5.1,0.566,5.1,NaN,70.0,NaN,5.0,2017/09/11 00:30,2017/09/10 16:30,NaN
2017-09-10 16:45:00-08:00,-0.604,13.9,9.1,94.0,NaN,NaN,4.9,5.0,0.566,4.9,NaN,52.0,NaN,3.0,2017/09/11 00:45,2017/09/10 16:45,NaN
2017-09-10 17:00:00-08:00,-0.601,13.9,9.1,94.0,NaN,NaN,5.1,5.0,0.566,5.1,NaN,57.0,NaN,3.0,2017/09/11 01:00,2017/09/10 17:00,NaN
2017-09-10 17:15:00-08:00,-0.594,13.9,9.1,93.0,NaN,NaN,5.2,5.2,0.566,5.2,NaN,67.0,NaN,3.0,2017/09/11 01:15,2017/09/10 17:15,NaN
2017-09-10 17:30:00-08:00,-0.582,13.8,10.2,93.0,NaN,NaN,5.2,5.1,0.566,5.0,NaN,90.0,NaN,3.0,2017/09/11 01:30,2017/09/10 17:30,NaN


In [31]:
wide_dat=wide_dat.reset_index(drop=True).copy() #drop index in output dataframe

In [32]:
save_dat=wide_dat[out_columns]

In [33]:
#directory to save output data
save_pth=r"Q:/Project Data/GlacierData/Benchmark_Program/Data/" +Glacier+ r"/AllYears/Wx/LVL0/emily/"

In [34]:
out_nm=Glacier.lower()+ Station + '_all_15minL0.csv'

In [35]:
save_dat.to_csv(os.path.join(save_pth, out_nm), index=False, float_format='%g')

In [36]:
#wide_dat[['TAspirated1', 'TAspirated2', 'Tpassive1']].plot()